In [39]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [9]:
df_horario = pd.DataFrame()
abas = ['01-2020 a 06-2020','07-2020 a 01-2021']

In [10]:
for aba in tqdm(abas):
    df_horario = df_horario.append(pd.read_excel(r'BD\Horário_Primeira-Entrada_Última-Saída_01-2020_01-2021.xlsx', sheet_name = aba))

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:47<00:00, 23.64s/it]


In [11]:
df_horario.head()

,MATRÍCULA,DATA,HORÁRIO INÍCIO,HORÁRIO FINAL,PRIMEIRA ENTRADA,ÚLTIMA SAÍDA
0,215695,2020-01-02,08:00,17:48,08:00,17:48
1,216322,2020-01-02,07:00,16:48,07:21,16:48
2,216328,2020-01-02,08:00,17:48,07:00,16:49
3,216617,2020-01-02,06:00,14:20,06:55,14:20
4,216864,2020-01-02,07:00,15:20,07:01,15:20


In [21]:
df_horario.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 781042 entries, 0 to 469026
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   MATRÍCULA         781042 non-null  int64         
 1   DATA              781042 non-null  datetime64[ns]
 2   HORÁRIO INÍCIO    781001 non-null  object        
 3   HORÁRIO FINAL     781001 non-null  object        
 4   PRIMEIRA ENTRADA  650361 non-null  object        
 5   ÚLTIMA SAÍDA      650382 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 41.7+ MB


In [45]:
df_noturno = df_horario.drop(['PRIMEIRA ENTRADA', 'ÚLTIMA SAÍDA'],axis=1)
df_noturno = df_noturno.sort_values('DATA', ascending=False)
df_noturno = df_noturno.drop_duplicates(['MATRÍCULA', 'HORÁRIO INÍCIO', 'HORÁRIO FINAL'])
df_noturno['HORÁRIO INÍCIO'] = df_noturno['HORÁRIO INÍCIO'].map(lambda x: float(str(x).replace(':','')))
df_noturno['HORÁRIO FINAL'] = df_noturno['HORÁRIO FINAL'].map(lambda x: float(str(x).replace(':','')))
df_noturno.head()

,MATRÍCULA,DATA,HORÁRIO INÍCIO,HORÁRIO FINAL
452906,236127,2021-01-31,1800.0,600.0
450216,230125,2021-01-31,1030.0,2230.0
451768,233129,2021-01-31,0.0,800.0
451259,235385,2021-01-31,800.0,1620.0
451258,235373,2021-01-31,1000.0,2200.0


In [46]:
df_noturno.shape

(11751, 4)

In [47]:
df_noturno['noturno (inicio)'] = df_noturno['HORÁRIO INÍCIO'].map(lambda x: 1 if (x >= 2200) | (x <= 500)  else 0)
df_noturno['noturno (fim)'] = df_noturno['HORÁRIO FINAL'].map(lambda x: 1 if (x >= 2200) | (x <= 500)  else 0)
df_noturno['NOTURNO'] = np.where(((df_noturno['noturno (inicio)'] == 1) | (df_noturno['noturno (fim)'] == 1)), 1, 0)
df_noturno = df_noturno[['MATRÍCULA', 'DATA', 'NOTURNO']]
df_noturno.head()

,MATRÍCULA,DATA,NOTURNO
452906,236127,2021-01-31,0
450216,230125,2021-01-31,1
451768,233129,2021-01-31,1
451259,235385,2021-01-31,0
451258,235373,2021-01-31,1


In [48]:
df_noturno.to_csv(r'BD Tratado\noturno.csv',sep=';',index=False)

In [69]:
df_atraso = df_horario.copy()

df_atraso['HORÁRIO INÍCIO'] = pd.to_datetime(df_atraso['DATA'].astype(str) + ' ' + df_atraso['HORÁRIO INÍCIO'])
df_atraso['HORÁRIO FINAL'] = pd.to_datetime(df_atraso['DATA'].astype(str) + ' ' + df_atraso['HORÁRIO FINAL'])
df_atraso['PRIMEIRA ENTRADA'] = pd.to_datetime(df_atraso['DATA'].astype(str) + ' ' + df_atraso['PRIMEIRA ENTRADA'])
df_atraso['ÚLTIMA SAÍDA'] = pd.to_datetime(df_atraso['DATA'].astype(str) + ' ' + df_atraso['ÚLTIMA SAÍDA'])

df_atraso.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 781042 entries, 0 to 469026
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   MATRÍCULA         781042 non-null  int64         
 1   DATA              781042 non-null  datetime64[ns]
 2   HORÁRIO INÍCIO    781001 non-null  datetime64[ns]
 3   HORÁRIO FINAL     781001 non-null  datetime64[ns]
 4   PRIMEIRA ENTRADA  650361 non-null  datetime64[ns]
 5   ÚLTIMA SAÍDA      650382 non-null  datetime64[ns]
dtypes: datetime64[ns](5), int64(1)
memory usage: 57.8 MB


In [89]:
df_atraso['Inicio - Atraso'] = np.where(df_atraso['HORÁRIO INÍCIO'] < df_atraso['PRIMEIRA ENTRADA'],
                                        (-(df_atraso['HORÁRIO INÍCIO'] - df_atraso['PRIMEIRA ENTRADA']).astype('timedelta64[m]')) ,0)
df_atraso['Inicio - Atraso'] = df_atraso['Inicio - Atraso'].map(lambda x: x if x > 15 else 0)
df_atraso['Final - Atraso'] = np.where(df_atraso['HORÁRIO FINAL'] > df_atraso['ÚLTIMA SAÍDA'],
                                        ((df_atraso['HORÁRIO FINAL'] - df_atraso['ÚLTIMA SAÍDA']).astype('timedelta64[m]')) ,0)
df_atraso['Inicio - Atraso'] = df_atraso['Inicio - Atraso'].map(lambda x: x if x > 15 else 0)
df_atraso.head()

,MATRÍCULA,DATA,HORÁRIO INÍCIO,HORÁRIO FINAL,PRIMEIRA ENTRADA,ÚLTIMA SAÍDA,Inicio - Atraso,Final - Atraso,ATRASO
0,215695,2020-01-02,2020-01-02 08:00:00,2020-01-02 17:48:00,2020-01-02 08:00:00,2020-01-02 17:48:00,0.0,0.0,0.0
1,216322,2020-01-02,2020-01-02 07:00:00,2020-01-02 16:48:00,2020-01-02 07:21:00,2020-01-02 16:48:00,21.0,0.0,21.0
2,216328,2020-01-02,2020-01-02 08:00:00,2020-01-02 17:48:00,2020-01-02 07:00:00,2020-01-02 16:49:00,0.0,59.0,59.0
3,216617,2020-01-02,2020-01-02 06:00:00,2020-01-02 14:20:00,2020-01-02 06:55:00,2020-01-02 14:20:00,55.0,0.0,55.0
4,216864,2020-01-02,2020-01-02 07:00:00,2020-01-02 15:20:00,2020-01-02 07:01:00,2020-01-02 15:20:00,0.0,0.0,1.0


In [90]:
df_atraso['ATRASO'] = df_atraso['Inicio - Atraso'] + df_atraso['Final - Atraso']
df_atraso.head()

,MATRÍCULA,DATA,HORÁRIO INÍCIO,HORÁRIO FINAL,PRIMEIRA ENTRADA,ÚLTIMA SAÍDA,Inicio - Atraso,Final - Atraso,ATRASO
0,215695,2020-01-02,2020-01-02 08:00:00,2020-01-02 17:48:00,2020-01-02 08:00:00,2020-01-02 17:48:00,0.0,0.0,0.0
1,216322,2020-01-02,2020-01-02 07:00:00,2020-01-02 16:48:00,2020-01-02 07:21:00,2020-01-02 16:48:00,21.0,0.0,21.0
2,216328,2020-01-02,2020-01-02 08:00:00,2020-01-02 17:48:00,2020-01-02 07:00:00,2020-01-02 16:49:00,0.0,59.0,59.0
3,216617,2020-01-02,2020-01-02 06:00:00,2020-01-02 14:20:00,2020-01-02 06:55:00,2020-01-02 14:20:00,55.0,0.0,55.0
4,216864,2020-01-02,2020-01-02 07:00:00,2020-01-02 15:20:00,2020-01-02 07:01:00,2020-01-02 15:20:00,0.0,0.0,0.0


In [92]:
df_atraso.to_csv(r'BD Tratado\atraso.csv',sep=';',index=False)